In [ ]:
import os, sys

In [ ]:
base_dir = os.path.split(os.getcwd())[0]
os.chdir(os.path.join(base_dir,'core'))

In [ ]:
import pickle as pk

import numpy as np
import cv2
import sys
import argparse
import time
import copy
import datetime

import matplotlib.pyplot as plt

from utils import ARUCO_DICT, display_pose
from posesingle import PoseSingle

In [ ]:
k_matrix = "../calib/calibration_matrix_hikvision.npy"
d_coeff = "../calib/distortion_coefficients_hikvision.npy"
m_dict = "../demo_tags/marker_poses.pk"
edge_len = 0.1645
marker_step = 2.095
n_markers = 4

type_ = "DICT_7X7_100"

if ARUCO_DICT.get(type_, None) is None:
    print(f"ArUCo tag type '{type_}' is not supported")
    sys.exit(0)

aruco_dict_type = ARUCO_DICT[type_]
calibration_matrix_path = k_matrix
distortion_coefficients_path = d_coeff
marker_dict_path = m_dict


k = np.load(calibration_matrix_path)
d = np.load(distortion_coefficients_path)

with open(marker_dict_path,'rb') as f:
    marker_dict = pk.load(f)

estimator = PoseSingle(aruco_dict_type,
                       n_markers,
                       marker_step,
                       edge_len,
                       k,
                       d)

In [ ]:
# Test on single image
image = "../data/hikvision/demo1/frames_r/3.png"

image = cv2.imread(image)
time.sleep(.1)

# output = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# plt.imshow(output)
# plt.show()

frame, pose, dt, size = estimator(image, "now", True)

output = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

output = display_pose(output, pose[:-1,[3]])

plt.imshow(output)
plt.show()
print(pose)

In [ ]:
# Test on multiple images
for i in range(318,320):
    print(i)
    image = cv2.imread(f"../data/hikvision/demo1/frames_f/{i}.png")
    time.sleep(.1)
    
    # output = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # plt.imshow(output)
    # plt.show()
    
    frame, pose, dt, size = estimator(image, "now", True)
    
    output = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    try:
        output = display_pose(output, pose[:-1,[3]])
    except: continue
    plt.imshow(output)
    plt.show()
    print(pose)
    

In [ ]:
frame.shape

In [ ]:
# Alter intrinsic matrix with resolutoin change
k_video = copy.deepcopy(k)
k_video[1,1] = k_video[1,1] * 720/960

# or

k_video = k

In [ ]:
# np.save("camera_matrix_video.npy", k_video)

In [ ]:
cap = cv2.VideoCapture('../data/hikvision/demo1/r.mp4')

In [ ]:
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)/10)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

estimator_video = PoseSingle(
    aruco_dict_type,
    n_markers,
    marker_step,
    edge_len,
    k,
    d
)

out = cv2.VideoWriter('../data/results/hikvision_r_weighted_flip.mov', cv2.VideoWriter_fourcc('M','J','P','G'), 30, (width,height))

In [ ]:
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        frame, pose, dt, size = estimator_video(frame, "now", True)
        # print(type(pose))
        output = frame#cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # print(pose.shape==())
        if pose.shape!=():
            output = display_pose(output, pose[:-1,[3]])
        
        out.write(output)
  # Break the loop
    else: 
        break

cap.release()
out.release()

In [ ]:
frame